## **1.2.1 Extractie attributendata uit vectorlaag op basis van puntdata**
Om de invloed van populatiedichtheid en bouwjaar op de waarnemingen te onderzoeken zijn er data verzameld uit [Statistische gegevens per vierkant 2015]( https://www.cbs.nl/nl-nl/dossier/nederland-regionaal/geografische-data/kaart-van-100-meter-bij-100-meter-met-statistieken) (Populatie dichtheid) en uit [Basisregistratie Adressen en Gebouwen (BAG)]( https://www.digitaleoverheid.nl/overzicht-van-alle-onderwerpen/stelsel-van-basisregistraties/10-basisregistraties/bag/) (bouwjaar).

'Statistische gegevens per vierkant 2015' bevat statistische gegevens van het CBS over demografie, wonen, energie, sociale zekerheid, dichtheid en nabijheid van voorzieningen gegeven in vakken van 100 bij 100 meter. De data zijn aangeleverd als attributen van een vector kaart. De data uit deze kaart laag zijn verzameld met een puntextractie script wat berekend binnen welk polygoon een coördinaat valt en uit dit polygoon de aangegeven attributen kopieert.

BAG is een dataset die informatie bevat over alle adressen en gebouwen in Nederland, zoals het bouwjaar. Deze gegevens worden door gemeenten centraal beschikbaar gesteld via de Landelijke Voorziening BAG (LV BAG). De data zijn aangeleverd in de vorm van een geodatabase met verschillende kaartlagen. Afgezien van de noodzaak om een specifieke kaartlaag te selecteren, is hetzelfde script dat in de voorgaande alinea wordt toegepast, ook hier van toepassing.



In [2]:
# Laad de benodigde ondersteunende bibliotheken in
from shapely.geometry import Point
from pyproj import CRS as ProjCRS
from pathlib import Path
import geopandas as gpd
import pandas as pd
import pyproj
import sys
import os

## Intro en settings
Deze code is bedoeld om data uit de attributentabel van polygons te linken aan puntdata in een Excel sheet. De puntdata moeten gegeven worden in Rijksdriehoekscoördinaten (RD-coördinaten) waarbij de X en Y component in losse kolommen gegeven wordt. Het gegeven Excel bestand hoeft veder niet leeg van data te zijn en kan zelfs meerdere tabbladen bevatten. Om dit te accommoderen wordt wel gevraagd om de naam van het tabblad en de kolommen met de coördinaten op voorhand te selecteren.

**Excel:**

In [3]:
# Vul hieronder het pad naar je Excel bestand in. 
# Laat hierbij het "Path(r"..")" intact.
ExcelVraag = Path(r"Input_data/TestInput.xlsx")

# Geef hier de naam van de Excel sheet met de coördinaten voor de extractie.
ExcelSheet = "Sheet1"

# Geef hier de Kolommen van de Excel sheet met coördinaten. 
# Dit script werkt voor RD coördinaten en gaat er vanuit dat de X en Y coördinaten in losse kolommen staan
ExcelX = "RDX"
ExcelY = "RDY"

# Defineer het coordinaten systeem van de observaties en van de rasterkaart
Waarnemingen_crs = pyproj.CRS("EPSG:28992")  # Amersfoort RD New

De polygonen en attributen tabel waar data uit verzameld word is gelinkt aan een kaart bestand in de vorm van een .shp of een .gdb (dit is een map en niet een los bestand). In dit laatste geval moet naast de locatie van de kaart ook het de specifieke kaart laag aangegeven worden. Deze kan gevonden worden door de .gdb in te laden in een GIS programma en de naam van het kaart component over te nemen.

Daarnaast is het nodig de kolomnaam van de data die gekopieerd gaat worden uit de attributen tabel te noteren. Het is mogelijk meerdere attributen van dezelfde kaart(laag) in een keer over te nemen. De notatie hiervoor is als volgt:

KaartKolommen = ["Kolomnaam 1", "Kolomnaam 2", "etc."]

**Kaart / Shape file:**

In [4]:
# Locatie van de kaart.shp of kaart.gdb
# Laat hierbij het "Path(r"..")" intact.
Kaart = Path(r"Input_data/basisregistratie_adressen_en_gebouwen_(BAG)_05-10-2021/BAG.gdb")

# Geef hier de naam van de kaartlaag als de geselecteerde kaart een .gdb bestand is
Kaartlaag = "Pand"

# De naam van de kolom(men) met de data die je wil linken
KaartKolommen = ["bouwjaar"]


## Het script

In [5]:
# Lees het kaart-bestand in. Als het niet van het goede bestandstype is word het script onderbroken.
if 'KaartGDF' not in globals():
    if Kaart.suffix == ".gdb":
        print(f"{Kaart.stem + Kaart.suffix} word ingelanden. afhankelijk van de bestandsgrote kan dit enkele minuten duren.")
        KaartGDF = gpd.read_file(Kaart, layer=Kaartlaag, engine="pyogrio", use_arrow=True)
    elif Kaart.suffix == ".shp":
        print(f"{Kaart.stem + Kaart.suffix} word ingelanden. afhankelijk van de bestandsgrote kan dit enkele minuten duren")
        KaartGDF = gpd.read_file(Kaart, engine="pyogrio")
    else:
        sys.exit(f"Script gestopt. Het kaart bestand is {Kaart.suffix}, dit moet type .gdb of .shp zijn")
else:
    print(f"{Kaart.stem + Kaart.suffix} ophalen uit memory...")

# Filtert de kaartdata zodat alleen de coördinaatdata en gewenste kolommen overblijven
KaartKolommenFilter = KaartKolommen
KaartKolommenFilter.append("geometry")
KaartFiltertGDF = KaartGDF.filter(KaartKolommenFilter)

# Lees het Excel-bestand
print("Excel data verwerken...")
ExcelData = pd.read_excel(ExcelVraag, sheet_name=ExcelSheet)

if Waarnemingen_crs != KaartFiltertGDF.crs:
    # Maakt een transformer
    transformer = pyproj.Transformer.from_crs(Waarnemingen_crs, KaartFiltertGDF.crs, always_xy=True)
    # Voert de transformatie uit en maakt nieuwe kolommen
    ExcelData[ExcelX], ExcelData[ExcelY] = transformer.transform(ExcelData[ExcelX], ExcelData[ExcelY])
    
# Maak een nieuwe kolom 'geometry' in df met behulp van de x en y coördinaten
ExcelData["CoördinatenCode"] = ExcelData.apply(lambda row: Point(row[ExcelX], row[ExcelY]), axis=1)

# Converteer de DataFrame naar een GeoDataFrame
ExcelGDF = gpd.GeoDataFrame(ExcelData, geometry="CoördinatenCode")

# Zorg ervoor dat beide GeoDataFrames dezelfde CRS gebruiken
ExcelGDF.set_crs(KaartFiltertGDF.crs, inplace=True)

# Voeg de polygon informatie toe aan de punten
print("puntdata en kaartlaag vergelijken...")
ResultaatGDF = gpd.sjoin(ExcelGDF, KaartFiltertGDF, predicate='within')

# verwijder onnodige kolommen
ResultaatGDF.drop(columns=["CoördinatenCode", "index_right"], axis=1, inplace=True)

# Sla het resultaat op in een nieuw Excel-bestand
print("Output opslaan...")
if not os.path.exists("Output"):
    os.makedirs("Output")
ResultaatGDF.to_excel(f"Output/Resultaat, {ExcelVraag.stem}['{ExcelSheet}'], {Kaart.stem}{KaartKolommen}.xlsx")
print(rf"Output locatie: {os.getcwd()}/Output/Resultaat, {ExcelVraag.stem}['{ExcelSheet}'], {Kaart.stem}{KaartKolommen}.xlsx")

BAG.gdb word ingelanden. afhankelijk van de bestandsgrote kan dit enkele minuten duren.
Excel data verwerken...
puntdata en kaartlaag vergelijken...
Output opslaan...
Output locatie: c:\Users\Sjoer\AAA-Python\Insecten in huis\IIH_Repo\Bijlage 1, Definitief\Scripts/Output/Resultaat, TestInput['Sheet1'], BAG['bouwjaar', 'geometry'].xlsx
